Analysis of the error of regression models depending on the corrugation

In [5]:
# -*- coding: utf-8 -*-
'Evaluate models and create JSON of metrics'

# -*- coding: utf-8 -*-
# IMPORTS
# additional packages
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import argparse
import os
import time
import glob
from tqdm import tqdm
import json
import sys
import pickle
USER = os.getenv('USER')
sys.path.append('../../.')

# torch packages
import torch
import torchvision
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data as data
from torch import optim
from torchvision import datasets, transforms, models
from torchvision.models import resnet50, ResNet50_Weights, efficientnet_b0, EfficientNet_B0_Weights
import torchvision.transforms.functional as TF

from torchmetrics import MetricCollection
from torchmetrics.classification import MultilabelRecall, MultilabelPrecision, MultilabelF1Score

import torch.nn.functional as F
from torch.utils.data import Dataset

print('PyTorch version:', torch.__version__)



# custom functions
from utils.all_utils import train_test_split, compute_pos_weights, train_for_epoch, save_config, validate, \
                            balance_beta_pos_weights, tanimoto_torch, validate_regression

from utils.models import resnet_10_chan, effnet_10_chan, eff_net_bias_warmer, regressor_from_checkpoint, \
                            AtomCountPredictor


from utils.dataloader import  QUAM_atom_regressor_10_imgs, dataset_ks_dict, parse_k1_to_k, parse_val_ks

from utils.evaluation import virtual_epoch_paths



class configuration:
    def __init__(self):
        # EXPERIMENT PARAMETERS
        self.experiment_name = 'debug_regression'
        self.n_fp = 1024  # number of fingerprints of the backbone model
        self.output_size = 10 # output size of regressor model
        self.ratio = 0.95  # train/test ratio
        self.seed = 42
        self.virtual_epochs = 10 # number of times we save the model per epoch. If equal to 1
                                # we only save at the end of each epoch.
        self.tqdm_flag = True

        # TRAINING PARAMETERS

        self.lr = 0.001  # learning rate
        self.dropout = 0.5
        # self.momentum = 0.9  # momentum of SGD optimizer
        self.weight_decay = 0  # L2 regularization constant
        self.batch_size = 50  # Training batch size
        self.test_batch_size = 50  # Test batch size
        self.epochs = 100  # Number of epochs
        self.bias_warmer = True # setting appropiate bias
        self.pos_weight_balancer = True #for bigger fingerprints, it helps balance precision and recall
        self.pos_weight_beta = 10
        # DATA AUGMENTATION PARAMETERS

        # Rotation
        self.rot_prob = 0.5  # prob of rotation in data augmentation
        self.max_deg = 180  # maximum degrees of rotation in data augmentation

        # Zoom
        self.zoom_prob = 0.7  # prob of applying zoom
        self.max_zoom = 0.7  # maximum zooming in/out

        # Translation
        self.shift_prob = 0.3  # probability of vertical or/and horizontal translation
        self.max_shift = 20  # translation

        # Shear
        self.shear_prob = 0.3  # probability of shearing
        self.max_shear = 10  # maximum shearing angle

        # Gaussian noise
        self.gauss_noise = 2 # std of gaussian noise

        # comments
        self.comments = 'It uses all K folders'

        # METRICS AND MODELS PATHS
        self.exp_path = os.path.join('./models', self.experiment_name)
        self.metrics_path = os.path.join(self.exp_path, 'metrics')
        self.models_path = os.path.join(self.exp_path, 'models')



## Create arguments object
args = configuration()
# Print experiment name
print('Experiment name:', args.experiment_name)
# Set random seed for reproducibility
torch.backends.cudnn.deterministic = True  # fix the GPU to deterministic mode
torch.manual_seed(args.seed)  # CPU seed
torch.cuda.manual_seed_all(args.seed)  # GPU seed
random.seed(args.seed)  # python seed for image transformation
np.random.seed(args.seed)

# Load data

data_path = '../../data/dataset/atoms_count_w_H_df.gz'
dataset_df = pd.read_pickle(data_path)

test_df = dataset_df[dataset_df['split'] == 'test']
test_k_df = parse_val_ks(test_df)






print('Finished')


PyTorch version: 2.2.1+cu121
Experiment name: debug_regression
Finished


In [6]:
loaded_data = np.load('../../data/calculations/atom_count_preds.npz')

loaded_test_predictions = loaded_data['test_predictions']
loaded_ground_truth = loaded_data['ground_truth']

In [7]:
print(loaded_test_predictions.shape, loaded_ground_truth.shape)

(279848, 10) (279848, 10)


In [8]:
test_predictions = loaded_test_predictions
ground_truth = loaded_ground_truth

### Compute classification accuracy and regression metrics


In [9]:
print(test_k_df.iloc[:10])
print(ground_truth[:10])
print(test_predictions[:10])

          CID   C  Br  Cl  F  I  N  O  P  S   H split  \
1    10001048  25   0   0  0  0  6  0  0  0  16  test   
7       10005   2   0   0  0  0  4  0  0  0   4  test   
14  100128716  15   0   0  0  0  3  0  0  0  21  test   
15  100175925  10   0   0  0  0  3  4  0  2   9  test   
16  100197007  18   0   0  0  0  3  4  0  2  13  test   
18  100197976  20   0   0  0  0  2  3  0  1  14  test   
23     228827   7   0   0  0  0  6  2  0  0   8  test   
24     228834  14   0   0  0  0  8  4  0  0   8  test   
25   22886611  11   0   0  0  0  0  1  0  1  12  test   
27   22886614  14   0   0  0  0  0  0  0  1  12  test   

                                                 path  
1   /scratch/dataset/quam/K-1/Conformer3D_CID_1000...  
7   /scratch/dataset/quam/K-1/Conformer3D_CID_1000...  
14  /scratch/dataset/quam/K-1/Conformer3D_CID_1001...  
15  /scratch/dataset/quam/K-1/Conformer3D_CID_1001...  
16  /scratch/dataset/quam/K-1/Conformer3D_CID_1001...  
18  /scratch/dataset/quam/K-1/Confor

In [10]:
(test_k_df[['C', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'H']]<1).sum()/len(test_k_df)

C     0.000036
Br    0.823697
Cl    0.758072
F     0.817633
I     0.959900
N     0.079147
O     0.215213
P     0.998867
S     0.752977
H     0.000386
dtype: float64

In [11]:
(ground_truth<1).sum(axis=0)/len(ground_truth)

array([3.57336840e-05, 8.23697150e-01, 7.58072239e-01, 8.17633144e-01,
       9.59899660e-01, 7.91465367e-02, 2.15213259e-01, 9.98867242e-01,
       7.52976616e-01, 3.85923787e-04])

#### Compute classification metrics

In [12]:
from sklearn.metrics import precision_recall_fscore_support

In [13]:
atom_list = ['C', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'H']
gt_class = (ground_truth>0)
pred_class = (test_predictions>0)
class_atom_dict = dict()
for i, atom in tqdm(enumerate(atom_list)):
    prec, recall, f1_score, _ = precision_recall_fscore_support(gt_class[:, i], pred_class[:, i], average='binary')
    class_atom_dict[atom] = {'precision': prec, 
                             'recall': recall, 
                             'f1_score': f1_score}

    

10it [00:00, 16.20it/s]


In [14]:
class_atom_dict

{'C': {'precision': 0.99996426618831,
  'recall': 0.9999964265039059,
  'f1_score': 0.9999803460875314},
 'Br': {'precision': 0.9993504780075912,
  'recall': 0.9979123596416555,
  'f1_score': 0.9986309010699254},
 'Cl': {'precision': 0.9986270630960465,
  'recall': 0.9991433171351344,
  'f1_score': 0.9988851234116701},
 'F': {'precision': 0.9984897815086494,
  'recall': 0.9975311061036544,
  'f1_score': 0.998010213583478},
 'I': {'precision': 0.995906744972415,
  'recall': 0.9973266797362323,
  'f1_score': 0.9966162065894925},
 'N': {'precision': 0.9994295183504931,
  'recall': 0.9993441961358018,
  'f1_score': 0.9993868554220607},
 'O': {'precision': 0.9990984181336344,
  'recall': 0.9990665737793745,
  'f1_score': 0.9990824957027559},
 'P': {'precision': 0.8884297520661157,
  'recall': 0.6782334384858044,
  'f1_score': 0.7692307692307693},
 'S': {'precision': 0.9979878984395356,
  'recall': 0.9973093781191685,
  'f1_score': 0.9976485229106221},
 'H': {'precision': 0.9999320827018217,

In [15]:

def create_latex_table(class_atom_dict):
    headers = ['Atom', 'Precision', 'Recall', 'F1 Score']
    rows = []

    for atom, scores in class_atom_dict.items():
        precision = "{:.4f}".format(scores['precision'])
        recall = "{:.4f}".format(scores['recall'])
        f1_score = "{:.4f}".format(scores['f1_score'])
        row = f"{atom} & {precision} & {recall} & {f1_score} \\\\"
        rows.append(row)

    latex_table = "\\begin{table}[h]\n\\centering\n\\caption{Atom Classification Scores}\n\\begin{tabular}{cccc}\n"
    latex_table += " & ".join(headers) + " \\\\\n"
    latex_table += "\\hline\n"
    latex_table += "\n".join(rows)
    latex_table += "\n\\end{tabular}\n\\end{table}"

    return latex_table
table = create_latex_table(class_atom_dict)
print(table)

\begin{table}[h]
\centering
\caption{Atom Classification Scores}
\begin{tabular}{cccc}
Atom & Precision & Recall & F1 Score \\
\hline
C & 1.0000 & 1.0000 & 1.0000 \\
Br & 0.9994 & 0.9979 & 0.9986 \\
Cl & 0.9986 & 0.9991 & 0.9989 \\
F & 0.9985 & 0.9975 & 0.9980 \\
I & 0.9959 & 0.9973 & 0.9966 \\
N & 0.9994 & 0.9993 & 0.9994 \\
O & 0.9991 & 0.9991 & 0.9991 \\
P & 0.8884 & 0.6782 & 0.7692 \\
S & 0.9980 & 0.9973 & 0.9976 \\
H & 0.9999 & 1.0000 & 1.0000 \\
\end{tabular}
\end{table}


#### Compute mse and pearson correlation for each atom

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import r_regression
atom_list = ['C', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'H']
reg_metrics_atom = dict()
for i, atom in enumerate(atom_list):
    mse = mean_squared_error(ground_truth[:, i], test_predictions[:, i])
    mae = mean_absolute_error(ground_truth[:, i], test_predictions[:, i])
    pearson_r = r_regression(ground_truth[:, i].reshape(-1, 1), test_predictions[:, i].reshape(-1, 1))
    reg_metrics_atom[atom] = {'mse': mse, 
                              'mae':mae, 
                              ''' Pearson's r''': pearson_r[0]}

    

/home/mgonzalez/.conda/envs/pytorch/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/mgonzalez/.conda/envs/pytorch/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/mgonzalez/.conda/envs/pytorch/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/mgonzalez/.conda/envs/pytorch/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vecto

In [11]:
reg_metrics_atom

{'C': {'mse': 0.013803923,
  'mae': 0.01150982,
  " Pearson's r": 0.9995503226680649},
 'Br': {'mse': 0.00072539377,
  'mae': 0.0006682199,
  " Pearson's r": 0.9983419914335004},
 'Cl': {'mse': 0.00095051597,
  'mae': 0.000814728,
  " Pearson's r": 0.9987663905346181},
 'F': {'mse': 0.0024227437,
  'mae': 0.0014793745,
  " Pearson's r": 0.9970752120528198},
 'I': {'mse': 0.00045381777,
  'mae': 0.00035376346,
  " Pearson's r": 0.9953695154896979},
 'N': {'mse': 0.01153126,
  'mae': 0.00885838,
  " Pearson's r": 0.997658606678373},
 'O': {'mse': 0.007343272,
  'mae': 0.0058067236,
  " Pearson's r": 0.9978331883561562},
 'P': {'mse': 0.000543152,
  'mae': 0.00050027156,
  " Pearson's r": 0.7854063683990198},
 'S': {'mse': 0.0022619423,
  'mae': 0.001775964,
  " Pearson's r": 0.9962425755826927},
 'H': {'mse': 0.016148051,
  'mae': 0.012145879,
  " Pearson's r": 0.9992578732293284}}

In [20]:
def create_table_latex_v2(data_dict):
    headers = ['Atom', 'MAE', "Pearson's r"]
    rows = []

    for atom, values in data_dict.items():
        mae = "{:.4f}".format(values['mae'])
        pearson_r = "{:.4f}".format(values[" Pearson's r"])
        row = f"{atom} & {mae} & {pearson_r} \\\\"
        rows.append(row)

    latex_table = "\\begin{table}[h]\n\\centering\n\\caption{Atom Regression Scores}\n\\begin{tabular}{ccc}\n"
    latex_table += " & ".join(headers) + " \\\\\n"
    latex_table += "\\hline\n"
    latex_table += "\n".join(rows)
    latex_table += "\n\\end{tabular}\n\\end{table}"

    return latex_table
table = create_table_latex_v2(reg_metrics_atom)
print(table)

\begin{table}[h]
\centering
\caption{Atom Regression Scores}
\begin{tabular}{ccc}
Atom & MAE & Pearson's r \\
\hline
C & 0.0115 & 0.9996 \\
Br & 0.0007 & 0.9983 \\
Cl & 0.0008 & 0.9988 \\
F & 0.0015 & 0.9971 \\
I & 0.0004 & 0.9954 \\
N & 0.0089 & 0.9977 \\
O & 0.0058 & 0.9978 \\
P & 0.0005 & 0.7854 \\
S & 0.0018 & 0.9963 \\
H & 0.0121 & 0.9993 \\
\end{tabular}
\end{table}


In [21]:
reg_metrics_atom

{'C': {'mse': 0.013803923,
  'mae': 0.01150982,
  " Pearson's r": 0.9995503226680649},
 'Br': {'mse': 0.00072539377,
  'mae': 0.0006682199,
  " Pearson's r": 0.9983400801930626},
 'Cl': {'mse': 0.00095051597,
  'mae': 0.000814728,
  " Pearson's r": 0.9987638843927912},
 'F': {'mse': 0.0024227437,
  'mae': 0.0014793745,
  " Pearson's r": 0.9970718153314084},
 'I': {'mse': 0.00045381777,
  'mae': 0.00035376346,
  " Pearson's r": 0.9953778918543514},
 'N': {'mse': 0.01153126,
  'mae': 0.00885838,
  " Pearson's r": 0.9976658766818135},
 'O': {'mse': 0.007343272,
  'mae': 0.0058067236,
  " Pearson's r": 0.9978421341892169},
 'P': {'mse': 0.000543152,
  'mae': 0.00050027156,
  " Pearson's r": 0.7854038516591924},
 'S': {'mse': 0.0022619423,
  'mae': 0.001775964,
  " Pearson's r": 0.9962548491054385},
 'H': {'mse': 0.016148051,
  'mae': 0.012145879,
  " Pearson's r": 0.9992569638280011}}